#**Homecraft**-**OpenAI**

In [ ]:
pip install elasticsearch OpenAI

In [ ]:
from elasticsearch import Elasticsearch
from openai import OpenAI, ChatCompletion

# Initialize Elasticsearch
es = Elasticsearch("https://diar-demo-cluster.es.asia-southeast1.gcp.elastic-cloud.com/", api_key="NE9XTWVZNEJXRjBsbHhqQUZkbks6cVNTUWJjMnVSTFMtZzlBczdGLUFkdw==")

# Test the connection
if es.ping():
    print("Connected to Elasticsearch cluster")
else:
    print("Unable to connect to Elasticsearch cluster")

In [ ]:
# Initialize OpenAI
openaikey = input("Enter your key: ")
client = OpenAI(api_key=openaikey)

In [45]:
# Search ElasticSearch index and return details on relevant products
def search_products(query_text):

    # Elasticsearch query (BM25) and kNN configuration for hybrid search
    query = {
        "bool": {
            "must": [{
                "match": {
                    "title": {
                        "query": query_text,
                        "boost": 1
                    }
                }
            }],
            "filter": [{
                "exists": {
                    "field": "title-vector"
                }
            }]
        }
    }

    knn = {
        "field": "title-vector",
        "k": 1,
        "num_candidates": 20,
        "query_vector_builder": {
            "text_embedding": {
                "model_id": "sentence-transformers__all-distilroberta-v1",
                "model_text": query_text
            }
        },
        "boost": 24
    }

    fields = ["title", "description", "url", "availability", "price", "brand", "product_id"]
    index = 'home-depot-product-catalog-vector'
    resp = es.search(index=index,
                     query=query,
                     knn=knn,
                     fields=fields,
                     size=5,
                     source=False)

    doc_list = resp['hits']['hits']
    body = resp['hits']['hits']
    url = ''
    for doc in doc_list:
        #body = body + doc['fields']['description'][0]
        url = url + "\n\n" +  doc['fields']['url'][0]

    return body, url

# Search ElasticSearch index and return body and URL for crawled docs
def search_docs(query_text):


    # Elasticsearch query (BM25) and kNN configuration for hybrid search
    query = {
        "bool": {
            "must": [{
                "match": {
                    "title": {
                        "query": query_text,
                        "boost": 1
                    }
                }
            }],
            "filter": [{
                "exists": {
                    "field": "title-vector"
                }
            }]
        }
    }

    knn = {
        "field": "title-vector",
        "k": 1,
        "num_candidates": 20,
        "query_vector_builder": {
            "text_embedding": {
                "model_id": "sentence-transformers__all-distilroberta-v1",
                "model_text": query_text
            }
        },
        "boost": 24
    }

    fields = ["title", "body_content", "url"]
    index = 'search-homecraft-ikea'
    resp = es.search(index=index,
                     query=query,
                     knn=knn,
                     fields=fields,
                     size=1,
                     source=False)

    body = resp['hits']['hits'][0]['fields']['body_content'][0]
    url = resp['hits']['hits'][0]['fields']['url'][0]

    return body, url

# Search ElasticSearch index for user's order history
def search_orders(user):

    # Use only text-search
    query = {
        "bool": {
            "must": [{
                "match": {
                    "user_id": {
                        "query": user,
                        "boost": 1
                    }
                }
            }]
        }
    }

    fields = ["id", "order_id", "user_id", "product_id" "status", "created_at", "shipped_at", "delivered_at", "returned_at", "sale_price"]
    index = 'bigquery-thelook-order-items'
    resp = es.search(index=index,
                     query=query,
                     fields=fields,
                     size=10,
                     source=False)

    order_items_list = resp['hits']['hits']

    return order_items_list

In [ ]:
query = input("Enter your query: ")

resp_products, url_products = search_products(query)
resp_docs, url_docs = search_docs(query)
resp_order_items = search_orders(1) # 1 is the hardcoded userid, to simplify this scenario. You should take user_id by user session

prompt = f"Answer this question: {query}.\n If product information is requested use the information provided in this JSON: {resp_products} listing the identified products in bullet points with this format: Product name, product key features, price, web url. \n For other questions use the documentation provided in these docs: {resp_docs} and your own knowledge. \n If the question contains requests for user past orders consider the following order list: {resp_order_items}"

# Generate response from ChatGPT
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[{"role": "system", "content": prompt}]
)

# Print the model's response
print(response.choices[0].message.content)